## Pre-requisites for this tutorial: 
* install Docker (https://docs.docker.com/get-docker/)
* have ~10Gb of free space to download the image
* set the token_dataset folder in the same directory as this notebook

This notebook will take you through the execution of the major pipeline stages for RABIES with a Docker installation (could be conducted similarly with a Singularity installation, see online documentation for alternative syntax https://rabies.readthedocs.io/en/latest/running_the_software.html). We will be using a 'token dataset', that is, fake images with very low resolution which are ideal for testing installation and execution rapidly. You can download the token dataset from https://github.com/grandjeanlab/INCF_preclinical/rabies/token_dataset, and make sure the data is in the same folder as this notebook.

Along with this tutorial (in https://github.com/grandjeanlab/INCF_preclinical/rabies) are included pre-generated QC outputs on a test dataset containing real images, which can be found online (**link**), as well as the notebook with the commands we ran to generate those outputs.

In [1]:
# Pull the RABIES image from Docker hub https://hub.docker.com/r/gabdesgreg/rabies. Here we select the version 0.4.8. 
#***this step can take a long time, do this ahead of the tutorial
!docker pull gabdesgreg/rabies:0.4.8

0.4.8: Pulling from gabdesgreg/rabies

7da0a88c: Already exists 
a5f15c7a: Already exists 
be87baa1: Already exists 
26eae209: Pulling fs layer 
48f25545: Pulling fs layer 
14b5f91a: Pulling fs layer 
9c6fce70: Pulling fs layer 
a348ca3c: Pulling fs layer 
b25c3815: Pulling fs layer 
7b94ca36: Pulling fs layer 
30beeafe: Pulling fs layer 
4d1f9573: Pulling fs layer 
b1324eac: Pulling fs layer 
2f65a254: Pulling fs layer 
492d75f6: Pulling fs layer 
f4c6fb92: Pulling fs layer 
c6fce70: Waiting fs layer 
25c3815: Waiting fs layer 
b94ca36: Waiting fs layer 
b1324eac: Waiting fs layer 
2f65a254: Waiting fs layer 
f4c6fb92: Waiting fs layer 
8f16a8b3: Waiting fs layer 
5683cf99: Waiting fs layer 
ce1ecc28: Pulling fs layer 
Digest: sha256:f5517c1d7accbd3a398a0fb3fabd210727667300f32fb0dc909a6aaae89667ceDownloading  186.6MB/683MBDownloading  275.8MB/277.4MBDownloading    874MB/1.34GB
Status: Downloaded newer image for gabdesgreg/rabies:0.4.8
docker.io/gabdesgreg/rabies:0.4.8


In [2]:
!docker run gabdesgreg/rabies:0.4.8 --help # test installation by printing the --help

usage: rabies [-h]
              [-p {Linear,MultiProc,SGE,SGEGraph,PBS,LSF,SLURM,SLURMGraph}]
              [--local_threads LOCAL_THREADS]
              [--scale_min_memory SCALE_MIN_MEMORY] [--min_proc MIN_PROC]
              [--verbose VERBOSE]
              Processing stage ...

RABIES performs multiple stages of rodent fMRI image processing, including preprocessing, 
confound correction, simple analyses and data quality assessment.

optional arguments:
  -h, --help            show this help message and exit

Processing options:
  The RABIES workflow is seperated into three main processing stages: preprocessing, 
  confound correction and analysis. Outputs from the preprocessing provide the inputs for
  the subsequent confound correction, and finally analysis.

  Processing stage      Description
    preprocess          
                        Conducts preprocessing on an input dataset in BIDS format. Preprocessing includes 
                        motion realignment, susceptibil

# Preprocessing

In [1]:
!docker run gabdesgreg/rabies:0.4.8 preprocess --help # print the parameters available for the preprocessing step

usage: rabies preprocess [-h] [--bold_only] [--anat_autobox] [--bold_autobox]
                         [--apply_despiking]
                         [--HMC_option {intraSubjectBOLD,0,1,2,3}]
                         [--apply_slice_mc] [--detect_dummy]
                         [--data_type {int16,int32,float32,float64}]
                         [--anat_inho_cor ANAT_INHO_COR]
                         [--anat_robust_inho_cor ANAT_ROBUST_INHO_COR]
                         [--bold_inho_cor BOLD_INHO_COR]
                         [--bold_robust_inho_cor BOLD_ROBUST_INHO_COR]
                         [--commonspace_reg COMMONSPACE_REG]
                         [--bold2anat_coreg BOLD2ANAT_COREG]
                         [--nativespace_resampling NATIVESPACE_RESAMPLING]
                         [--commonspace_resampling COMMONSPACE_RESAMPLING]
                         [--anatomical_resampling ANATOMICAL_RESAMPLING]
                         [--apply_STC] [--TR TR]
                         [--tp

### Execution of preprocessing:
* the `-v` arguments set the path between your local computed and inside the container as follow: `-v /local_path:/path_inside_container`. The /local_path must point to the input dataset and/or output folder, and you can provide any sensible name for the /path_inside_container. 
* `-p MultiProc` will execute the pipeline in parallel to maximize efficiency, using local resources available on your computer
* `--local_threads` can set the maximum number of parallel threads (since by default all available threads are selected, and this may slow down your computer for other tasks). I select 6, as there are 12 on my computer
* the other parameters are selected for the sake of executing the token dataset quickly

In [2]:
# create the output folder,and allow for writing permissions so that Docker can write new outputs inside
!mkdir -p token_rabies_outputs
!chmod 777 token_rabies_outputs

### below are extra parameters that are necessary to process the token dataset effectively, do not use these with real data!
!docker run -it --rm \
-v $PWD/token_dataset:/token_dataset:ro \
-v $PWD/token_rabies_outputs:/token_rabies_outputs/ \
gabdesgreg/rabies:0.4.8 -p MultiProc --local_threads 6 \
preprocess /token_dataset/ /token_rabies_outputs \
--anatomical_resampling 1x1x1 \
--anat_inho_cor method=disable,otsu_thresh=2,multiotsu=false --bold_inho_cor method=disable,otsu_thresh=2,multiotsu=false \
--bold2anat_coreg registration=no_reg,masking=true,brain_extraction=true \
--commonspace_reg masking=false,brain_extraction=false,fast_commonspace=true,template_registration=no_reg --data_type int16


230914-18:51:24,866 nipype.workflow INFO:
	 Running RABIES - version: 0.4.8
230914-18:51:24,867 nipype.workflow INFO:
	 CLI INPUTS: 
-> rabies_stage = preprocess 
-> plugin = MultiProc 
-> local_threads = 6 
-> scale_min_memory = 1.0 
-> min_proc = 1 
-> verbose = 1 
-> bids_dir = /token_dataset 
-> output_dir = /token_rabies_outputs 
-> bold_only = False 
-> anat_autobox = False 
-> bold_autobox = False 
-> apply_despiking = False 
-> HMC_option = intraSubjectBOLD 
-> apply_slice_mc = False 
-> detect_dummy = False 
-> data_type = int16 
-> anat_inho_cor = {'method': 'disable', 'otsu_thresh': '2', 'multiotsu': False} 
-> anat_robust_inho_cor = {'apply': False, 'masking': False, 'brain_extraction': False, 'template_registration': 'SyN'} 
-> bold_inho_cor = {'method': 'disable', 'otsu_thresh': '2', 'multiotsu': False} 
-> bold_robust_inho_cor = {'apply': False, 'masking': False, 'brain_extraction': False, 'template_registration': 'SyN'} 
-> commonspace_reg = {'masking': False, 'brain_ex

In [6]:
# inspect the outputs (2 layers deep)
!tree -L 2 token_rabies_outputs

token_rabies_outputs
├── anat_datasink
│   └── anat_preproc
├── boilerplate.txt
├── bold_datasink
│   ├── commonspace_bold
│   ├── commonspace_CSF_mask
│   ├── commonspace_labels
│   ├── commonspace_mask
│   ├── commonspace_resampled_template
│   ├── commonspace_vascular_mask
│   ├── commonspace_WM_mask
│   ├── inho_cor_bold
│   ├── inho_cor_bold_warped2anat
│   ├── initial_bold_ref
│   ├── input_bold
│   ├── native_bold
│   ├── native_bold_ref
│   ├── native_brain_mask
│   ├── native_CSF_mask
│   ├── native_labels
│   ├── native_WM_mask
│   ├── raw_brain_mask
│   ├── std_map_preprocess
│   └── tSNR_map_preprocess
├── confounds_datasink
│   ├── confounds_csv
│   ├── FD_csv
│   ├── FD_voxelwise
│   └── pos_voxelwise
├── main_wf
│   ├── anat_inho_cor_wf
│   ├── bold_main_wf
│   ├── commonspace_reg_wf
│   ├── d3.js
│   ├── graph1.json
│   ├── graph.json
│   ├── index.html
│   ├── resample_template
│   ├── _scan_info_subject_idtoken2.sessionNone_split_name_sub-token2_T1w
│   ├── _scan_info

# Confound correction

In [3]:
!docker run gabdesgreg/rabies:0.4.8 confound_correction --help # print the parameters available for the confound correction step

usage: rabies confound_correction [-h] [--nativespace_analysis]
                                  [--image_scaling {None,background_noise,global_variance,voxelwise_standardization,grand_mean_scaling,voxelwise_mean}]
                                  [--detrending_order {linear,quadratic}]
                                  [--conf_list [{WM_signal,CSF_signal,vascular_signal,global_signal,aCompCor,mot_6,mot_24,mean_FD} ...]]
                                  [--frame_censoring FRAME_CENSORING]
                                  [--TR TR] [--highpass HIGHPASS]
                                  [--lowpass LOWPASS]
                                  [--edge_cutoff EDGE_CUTOFF]
                                  [--smoothing_filter SMOOTHING_FILTER]
                                  [--match_number_timepoints]
                                  [--ica_aroma ICA_AROMA] [--read_datasink]
                                  [--timeseries_interval TIMESERIES_INTERVAL]
                                 

In [7]:
# the command for confound correction should be customized. In this example, we only regress 6 motion parameters
!docker run -it --rm \
-v $PWD/token_dataset:/token_dataset:ro \
-v $PWD/token_rabies_outputs:/token_rabies_outputs/ \
gabdesgreg/rabies:0.4.8 -p MultiProc --local_threads 6 \
confound_correction /token_rabies_outputs/ /token_rabies_outputs --conf_list mot_6

230914-18:56:08,166 nipype.workflow INFO:
	 Running RABIES - version: 0.4.8
230914-18:56:08,166 nipype.workflow INFO:
	 CLI INPUTS: 
-> rabies_stage = confound_correction 
-> plugin = MultiProc 
-> local_threads = 6 
-> scale_min_memory = 1.0 
-> min_proc = 1 
-> verbose = 1 
-> preprocess_out = /token_rabies_outputs 
-> output_dir = /token_rabies_outputs 
-> nativespace_analysis = False 
-> image_scaling = global_variance 
-> detrending_order = linear 
-> conf_list = ['mot_6'] 
-> frame_censoring = {'FD_censoring': False, 'FD_threshold': 0.05, 'DVARS_censoring': False, 'minimum_timepoint': 3} 
-> TR = auto 
-> highpass = None 
-> lowpass = None 
-> edge_cutoff = 0 
-> smoothing_filter = None 
-> match_number_timepoints = False 
-> ica_aroma = {'apply': False, 'dim': 0, 'random_seed': 1} 
-> read_datasink = False 
-> timeseries_interval = all 

230914-18:56:09,535 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.6.0
230914-18:56:09,577 nipype

In [11]:
# a new datasink with cleaned timeseries was generated
!tree -L 1 token_rabies_outputs/confound_correction_datasink

token_rabies_outputs/confound_correction_datasink
├── cleaned_timeseries
└── plot_CR_overfit

2 directories, 0 files


# Analysis

In [8]:
!docker run gabdesgreg/rabies:0.4.8 analysis --help # print the parameters available for the analysis step

usage: rabies analysis [-h] [--scan_list [SCAN_LIST ...]]
                       [--prior_maps PRIOR_MAPS]
                       [--prior_bold_idx [PRIOR_BOLD_IDX ...]]
                       [--prior_confound_idx [PRIOR_CONFOUND_IDX ...]]
                       [--data_diagnosis]
                       [--outlier_threshold OUTLIER_THRESHOLD]
                       [--seed_list [SEED_LIST ...]]
                       [--seed_prior_list [SEED_PRIOR_LIST ...]] [--FC_matrix]
                       [--ROI_type {parcellated,voxelwise}]
                       [--ROI_csv ROI_CSV] [--group_ica GROUP_ICA] [--DR_ICA]
                       [--NPR_temporal_comp NPR_TEMPORAL_COMP]
                       [--NPR_spatial_comp NPR_SPATIAL_COMP]
                       [--network_weighting {absolute,relative}]
                       confound_correction_out output_dir

positional arguments:
  confound_correction_out
                        path to RABIES confound correction output directory.
           

In [12]:
# in this example, we will generate the data diagnosis output files, which uses dual regression for connectivity analysis if nothing else is specified
!docker run -it --rm \
-v $PWD/token_dataset:/token_dataset:ro \
-v $PWD/token_rabies_outputs:/token_rabies_outputs/ \
gabdesgreg/rabies:0.4.8 -p MultiProc --local_threads 6 \
analysis /token_rabies_outputs/ /token_rabies_outputs --data_diagnosis

230914-18:59:31,388 nipype.workflow INFO:
	 Running RABIES - version: 0.4.8
230914-18:59:31,388 nipype.workflow INFO:
	 CLI INPUTS: 
-> rabies_stage = analysis 
-> plugin = MultiProc 
-> local_threads = 6 
-> scale_min_memory = 1.0 
-> min_proc = 1 
-> verbose = 1 
-> confound_correction_out = /token_rabies_outputs 
-> output_dir = /token_rabies_outputs 
-> scan_list = ['all'] 
-> prior_maps = /home/rabies/.local/share/rabies/melodic_IC.nii.gz 
-> prior_bold_idx = [5, 12, 19] 
-> prior_confound_idx = [0, 1, 2, 6, 7, 8, 9, 10, 11, 13, 14, 21, 22, 24, 26, 28, 29] 
-> data_diagnosis = True 
-> outlier_threshold = 3.5 
-> seed_list = [] 
-> seed_prior_list = [] 
-> FC_matrix = False 
-> ROI_type = parcellated 
-> ROI_csv = /home/rabies/.local/share/rabies/DSURQE_40micron_labels.nii.gz 
-> group_ica = {'apply': False, 'dim': 0, 'random_seed': 1} 
-> DR_ICA = False 
-> NPR_temporal_comp = -1 
-> NPR_spatial_comp = -1 
-> network_weighting = absolute 

/home/rabies/miniforge/lib/python3.9/sit

In [15]:
# a new datasink with the connectivity outputs was generated
!tree -L 1 token_rabies_outputs/analysis_datasink
# as well as a datasink for the data diagnosis
!tree -L 1 token_rabies_outputs/data_diagnosis_datasink

token_rabies_outputs/analysis_datasink
├── dual_regression_nii
└── dual_regression_timecourse_csv

2 directories, 0 files
token_rabies_outputs/data_diagnosis_datasink
├── analysis_QC
├── corrected_CR_std_nii
├── CR_prediction_std_nii
├── figure_spatial_diagnosis
├── figure_temporal_diagnosis
├── GS_cov_nii
├── random_CR_std_nii
├── spatial_VE_nii
├── temporal_info_csv
└── temporal_std_nii

10 directories, 0 files


You've now executed every main stage of the pipeline using the token dataset, and explored the outputs generated by RABIES. If this worked smoothly, you may now attempt running the pipeline on the test dataset (**link**), or on your own data!